# DANIEL SILVEIRA DANTAS
# DSD@CESAR.SCHOOL
# LINK: https://www.kaggle.com/datasets/mloey1/ahcd1

### Imports:

In [147]:
import kagglehub
import os
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import copy

### Carregamento do dataset:

In [148]:
# Baixar dataset
base_dir = kagglehub.dataset_download("mloey1/ahcd1")
print("Baixado em:", base_dir)

csv_dir = os.path.join(base_dir, "Arabic Handwritten Characters Dataset CSV")
print("Pasta CSV:", csv_dir)

# Carregar CSVs (imagens e rótulos)
train_images = pd.read_csv(os.path.join(csv_dir, "csvTrainImages 13440x1024.csv"), header=None)
train_labels = pd.read_csv(os.path.join(csv_dir, "csvTrainLabel 13440x1.csv"), header=None)

test_images  = pd.read_csv(os.path.join(csv_dir, "csvTestImages 3360x1024.csv"), header=None)
test_labels  = pd.read_csv(os.path.join(csv_dir, "csvTestLabel 3360x1.csv"), header=None)

print("train_images:", train_images.shape)
print("train_labels:", train_labels.shape)
print("test_images:",  test_images.shape)
print("test_labels:",  test_labels.shape)


Using Colab cache for faster access to the 'ahcd1' dataset.
Baixado em: /kaggle/input/ahcd1
Pasta CSV: /kaggle/input/ahcd1/Arabic Handwritten Characters Dataset CSV
train_images: (13440, 1024)
train_labels: (13440, 1)
test_images: (3360, 1024)
test_labels: (3360, 1)


### Preparação dos Dados: Normalização e Ajuste dos Labels

In [149]:
# Converte para float32 e normaliza para [0,1]
X_train = torch.tensor(train_images.values, dtype=torch.float32) / 255.0
X_test  = torch.tensor(test_images.values,  dtype=torch.float32) / 255.0

# Labels são de 1 a 28; CrossEntropyLoss espera 0..27
y_train = torch.tensor(train_labels.values.squeeze() - 1, dtype=torch.long)
y_test  = torch.tensor(test_labels.values.squeeze()  - 1, dtype=torch.long)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape, "labels únicos:", torch.unique(y_train))
print("X_test:",  X_test.shape)
print("y_test:",  y_test.shape,  "labels únicos:", torch.unique(y_test))


X_train: torch.Size([13440, 1024])
y_train: torch.Size([13440]) labels únicos: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27])
X_test: torch.Size([3360, 1024])
y_test: torch.Size([3360]) labels únicos: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27])


### Construção dos Datasets e DataLoaders

In [150]:
batch_size = 128

train_dataset = TensorDataset(X_train, y_train)
test_dataset  = TensorDataset(X_test,  y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

# Checagem rápida
imgs, lbls = next(iter(train_loader))
print("Batch imagens:", imgs.shape)  # [batch, 1024]
print("Batch labels:", lbls.shape)


Batch imagens: torch.Size([128, 1024])
Batch labels: torch.Size([128])


### Definindo o modelo da rede:

In [151]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

NUM_CLASSES = 28  # 28 letras do alfabeto árabe

class MLP_AHCD(nn.Module):
    def __init__(self, num_classes):
        super(MLP_AHCD, self).__init__()

        # Entrada: 1024 pixels (32x32 achatado)
        self.fc1 = nn.Linear(32 * 32, 56)   # 1024 -> 28
        self.fc2 = nn.Linear(56, 56)        # 28 -> 28   (2ª camada oculta)
        self.fc3 = nn.Linear(56, num_classes)  # 28 -> 28 classes

        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten (batch_size, 1024)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))    # nova camada oculta
        x = self.fc3(x)
        return x

model = MLP_AHCD(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05e-1, weight_decay=3.625e-4)

scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

model


Device: cuda


MLP_AHCD(
  (fc1): Linear(in_features=1024, out_features=56, bias=True)
  (fc2): Linear(in_features=56, out_features=56, bias=True)
  (fc3): Linear(in_features=56, out_features=28, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

### Treinamento:

In [152]:

# ----- EARLY STOPPING CONFIG -----
patience = 10
best_test_acc = 0.0
best_epoch = 0
epochs_no_improve = 0
best_model_wts = copy.deepcopy(model.state_dict())

num_epochs = 100

for epoch in range(num_epochs):

    # --------- TREINO ---------
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, preds = torch.max(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # --------- TESTE ---------
    model.eval()
    test_correct = 0
    test_total = 0
    test_running_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_running_loss += loss.item() * labels.size(0)

            _, preds = torch.max(outputs, dim=1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = test_correct / test_total
    test_loss = test_running_loss / test_total

    print(f"Época {epoch+1}/{num_epochs} | "
          f"Treino Acc: {train_acc:.4f} | "
          f"Teste Acc: {test_acc:.4f} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Test Loss: {test_loss:.4f}")

    # --------- EARLY STOPPING CHECK ---------
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1
        epochs_no_improve = 0
        best_model_wts = copy.deepcopy(model.state_dict())
    else:
        epochs_no_improve += 1

    # --------- ATUALIZA O LEARNING RATE DO SCHEDULER ---------
    scheduler.step()

    # --------- VERIFICA SE PARA ---------
    if epochs_no_improve >= patience:
        print(f"\nEarly stopping ativado na época {epoch+1}.")
        print(f"Melhor Test Acc = {best_test_acc:.4f} (época {best_epoch})")
        break

# --------- CARREGA O MELHOR MODELO ---------
model.load_state_dict(best_model_wts)
print(f"\nModelo final carregado da época {best_epoch} "
      f"com Test Acc = {best_test_acc:.4f}")


Época 1/100 | Treino Acc: 0.2597 | Teste Acc: 0.4024 | Train Loss: 2.3997 | Test Loss: 1.7989
Época 2/100 | Treino Acc: 0.4953 | Teste Acc: 0.5482 | Train Loss: 1.5025 | Test Loss: 1.3119
Época 3/100 | Treino Acc: 0.6080 | Teste Acc: 0.6033 | Train Loss: 1.1357 | Test Loss: 1.1342
Época 4/100 | Treino Acc: 0.6731 | Teste Acc: 0.6360 | Train Loss: 0.9378 | Test Loss: 1.0329
Época 5/100 | Treino Acc: 0.7115 | Teste Acc: 0.6562 | Train Loss: 0.8120 | Test Loss: 0.9961
Época 6/100 | Treino Acc: 0.7411 | Teste Acc: 0.6735 | Train Loss: 0.7252 | Test Loss: 0.9538
Época 7/100 | Treino Acc: 0.7675 | Teste Acc: 0.6926 | Train Loss: 0.6600 | Test Loss: 0.9203
Época 8/100 | Treino Acc: 0.7894 | Teste Acc: 0.7036 | Train Loss: 0.5981 | Test Loss: 0.8838
Época 9/100 | Treino Acc: 0.8009 | Teste Acc: 0.6979 | Train Loss: 0.5666 | Test Loss: 0.9192
Época 10/100 | Treino Acc: 0.8134 | Teste Acc: 0.6929 | Train Loss: 0.5296 | Test Loss: 0.9082
Época 11/100 | Treino Acc: 0.8224 | Teste Acc: 0.7119 | Tra

### Conclusão:

### O processo de desenvolvimento e ajuste da rede MLP para reconhecer o alfabeto árabe foi longo e bastante experimental, principalmente por causa de um erro cometido logo no início — algo que só ficou claro no final e que alterou toda a trajetória dos testes. A ordem das modificações seguiu uma sequência lógica, mas muitas vezes precisei voltar atrás e revisar decisões anteriores. Comecei alterando o número de neurônios por camada, depois testei diferentes quantidades de camadas e, em seguida, aumentei o número de épocas para 20. Com esses primeiros experimentos, concluí prematuramente que a melhor configuração era com 2 camadas ocultas e 28 neurônios cada, pois, ao testar 3 camadas ocultas e/ou 56 neurônios com apenas 20 épocas, a rede apresentou underfitting e acabei assumindo, de forma equivocada, que 56 neurônios eram inadequados.
### Para tentar extrair mais aprendizado, aumentei o número de épocas para 100. Quando isso não foi suficiente, comecei a ajustar o learning rate, explorando valores maiores e menores até encontrar um valor adequado para a configuração que eu acreditava ser a correta (2 camadas e 28 neurônios). Em seguida, passei ao weight decay, testando diferentes valores para reduzir overfitting. Depois disso, ajustei o dropout, o que trouxe melhora, e só então alterei o batch size, que acabou ficando ótimo em 128. Com esses elementos estabilizados, adicionei um learning rate scheduler (StepLR), ajustei seu gamma e implementei o early stopping.
### Mesmo seguindo essa sequência estruturada, o processo esteve longe de ser direto. Houve momentos claros de overfitting, especialmente quando a acurácia de treino superava 0.90 enquanto a acurácia de teste permanecia por volta de 0.60 — o que exigiu ajustes cuidadosos de regularização e dropout. A busca pelos valores ótimos de cada parâmetro sempre envolveu aumentá-los e diminuí-los repetidamente em torno do valor inicial, mantendo todos os outros parâmetros fixos para isolar o efeito de cada mudança. Testei também outras ativações, mas todas pioraram o modelo, reforçando que ReLU era a opção mais adequada.
### No fim do processo, depois de ajustar praticamente todos os hiperparâmetros, percebi que havia cometido um erro inicial: eu nunca tinha testado 56 neurônios com 100 épocas, somente com 20. Ao refazer esse teste — o último da sequência — obtive de longe a melhor acurácia de teste, ultrapassando 0.70. Isso mostrou que todos os valores ótimos anteriores não eram os verdadeiros valores ótimos para essa rede, já que eles haviam sido encontrados para uma arquitetura inadequada (28 neurônios), e não para a arquitetura realmente mais promissora (56 neurônios). Concluí, assim, que o número de camadas, de neurônios e de épocas são fatores fundamentais para a qualidade final do modelo, e que eles devem ser sempre revisados com prioridade quando as métricas principais (perda e acurácia de teste para este trabalho) não estiverem convergindo para os valores desejados.